In [1]:
from utils import marich, dataset
from nets import *
from tqdm import tqdm
import numpy as np
import torch
import pickle
from sklearn.model_selection import train_test_split

Files already downloaded and verified


In [2]:
logreg = LogisticRegression(784,10).cuda()
logreg.load_state_dict(torch.load("./targets/logreg_mnist.pt"))

<All keys matched successfully>

In [3]:
# emnist = datasets.EMNIST('./emnist/train/', download=True, train=True, split = "letters", transform=transform)
# emnist,_ = random_split(emnist, [50000,74800])

In [4]:
# unlab_x = []
# unlab_y = []
# for j,k in emnist:
#     unlab_x.append(j)
#     unlab_y.append(torch.argmax(logreg(j.to(device).view(j.size(0),-1))).cpu())

In [5]:
# unlab_x = torch.stack(unlab_x)
# unlab_y = torch.tensor(unlab_y)

In [6]:
# with open('./imagenet_target/emnist_lr_x.pkl', 'wb') as f:
#   pickle.dump(unlab_x, f)

# with open('./imagenet_target/emnist_lr_y.pkl', 'wb') as f:
#   pickle.dump(unlab_y, f)

In [7]:
with open('./imagenet_target/emnist_lr_x.pkl', 'rb') as f:
  unlab_x = pickle.load(f)

with open('./imagenet_target/emnist_lr_y.pkl', 'rb') as f:
  unlab_y = pickle.load(f)

In [8]:
class basic_dataset(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x

class dataset2(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x
    def get_data(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        return x
    def get_dataset(self, idx):
        x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        return basic_dataset(x,y)
    def get_label(self,idx):
        y = torch.tensor(self.Y)[idx]
        return y
    def get_data_label_loader(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        
        return_set = basic_dataset(x,y)
        return torch.utils.data.DataLoader(return_set, batch_size = 64)

In [9]:
train_id, val_id = train_test_split(range(50000), test_size = 0.2)

In [10]:
unlab_dataset_train = dataset2(unlab_x[train_id], unlab_y[train_id])
unlab_dataset_val = dataset2(unlab_x[val_id], unlab_y[val_id])

In [11]:
testset = datasets.MNIST('../mnist/train/', download=True, train=False, transform=transform)
# trainset, testset = torch.utils.data.random_split(trainset, [50000, 10000])
# valset = datasets.MNIST('../mnist/test/', download=True, train=False, transform=transform)
testloader = DataLoader(testset, batch_size = 256)
validloader = DataLoader(unlab_dataset_val, batch_size = 256)

In [14]:
# kl_marich = []
# dist_marich = []
# agree_marich = []
# acc_marich = []

for i in tqdm(range(8,10)):
  log_attack = LogisticRegression(784,10)
  tl_log, vl_log, tal_log, samp_log = marich(log_attack, unlab_dataset_train, validloader, testloader, budget = 250, init_points = 300, rounds = 10, epochs = 10, LR = 0.02, model_name = "emnist_log"+str(i)+".pt", sampling = "all_egl", device = "cuda")
  
  acc_marich.append(tal_log)
  # kl_marich.append(kl_log)
  # dist_marich.append(dist)
  # agree_marich.append(agree)
  np.save("./results/acc_marich_lr_emnist.npy", np.array(acc_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/acc_marich.npy", np.array(acc_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/kl_marich.npy", np.array(kl_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/dist_marich.npy", np.array(dist_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/agree_marich.npy", np.array(agree_marich))
np.save("./results/samp_lr_emnist.npy", np.array(samp_log))

  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\pkpra\AppData\Local\Temp/ipykernel_14760/108530465.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]


Using CUDA
Test accuracy:  8.32
Training samples:  300
Epoch:  1
Train loss:  2.2764904499053955
Validation loss:  2.2073332548141478
Epoch time -----  0.6639907360076904  sec
Epoch:  2
Train loss:  2.1754884243011476
Validation loss:  2.1002582907676697
Epoch time -----  0.6663820743560791  sec
Epoch:  3
Train loss:  2.0784732341766357
Validation loss:  2.041681948304176
Epoch time -----  0.6662769317626953  sec
Epoch:  4
Train loss:  2.0164656162261965
Validation loss:  2.0020285189151763
Epoch time -----  0.6660115718841553  sec
Epoch:  5
Train loss:  1.9630276918411256
Validation loss:  1.9611162215471267
Epoch time -----  0.6672046184539795  sec
Epoch:  6
Train loss:  1.9272534608840943
Validation loss:  1.9403826594352722
Epoch time -----  0.650808572769165  sec
Epoch:  7
Train loss:  1.9059816598892212
Validation loss:  1.919881471991539
Epoch time -----  0.6653671264648438  sec
Epoch:  8
Train loss:  1.8827941179275514
Validation loss:  1.9101383358240127
Epoch time -----  0.65

C:\Users\pkpra\AppData\Local\Temp/ipykernel_14760/108530465.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
e:\pratik\new_marich\utils.py:384: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(probs).detach().cpu().numpy()


Using gradient sampling on  394  points
Using loss sampling on  316  points
Budget:  554
Training samples:  554
Training


C:\Users\pkpra\AppData\Local\Temp/ipykernel_14760/108530465.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
C:\Users\pkpra\AppData\Local\Temp/ipykernel_14760/108530465.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
e:\pratik\new_marich\nets.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(outputs)
e:\pratik\new_marich\u

...

Budget:  2954
Training samples:  2954
Training
Epoch:  1
Train loss:  1.7792484024737745


c:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Validation loss:  1.5819733530282973
Epoch time -----  0.7097575664520264  sec
Epoch:  2
Train loss:  1.7671609077047794
Validation loss:  1.5798051625490188
Epoch time -----  0.6688041687011719  sec
validation loss minimum, saving model
Epoch:  3
Train loss:  1.7625338194218088
Validation loss:  1.577900266647339
Epoch time -----  0.6671490669250488  sec
validation loss minimum, saving model
Epoch:  4
Train loss:  1.7599776907170073
Validation loss:  1.5781805336475372
Epoch time -----  0.6924688816070557  sec
Epoch:  5
Train loss:  1.7608909556206236
Validation loss:  1.5768379986286163
Epoch time -----  0.6682877540588379  sec
validation loss minimum, saving model
Epoch:  6
Train loss:  1.75973634770576
Validation loss:  1.5780759662389756
Epoch time -----  0.6508595943450928  sec
Epoch:  7
Train loss:  1.7537849786433768
Validation loss:  1.5765389561653138
Epoch time -----  0.6440122127532959  sec
validation loss minimum, saving model
Epoch:  8
Train loss:  1.7508933772432043
Vali

100%|██████████| 2/2 [05:53<00:00, 176.93s/it]

Test accuracy:  72.06
